# Setting Up Environment

## Installing Libraries

In [1]:

!pip install -q findspark

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=cad6188922796360dba70fb4ecc545cc38cc5e907417ab5e8452bc675f99dda9
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


## Importing Libraries

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [5]:
!jupyter labectension install jupyterlab-plotly

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir] [--paths] [--json]
               [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: bundlerextension console dejavu execute kernel kernelspec migrate nbclassic
nbconvert nbextension notebook run server serverextension troubleshoot trust

Jupyter command `jupyter-labectension` not found.


In [10]:
!pip install plotly
!pip install -U kaleido

In [7]:
from pyspark.sql.functions import count, col
import plotly.express as px
from pyspark.sql.functions import desc
from pyspark.sql.functions import desc
from pyspark.sql import functions as F
from pyspark.sql.window import Window



# Data Analysis

## Loading Data

In [9]:
df = spark.read.load("gs://discog-bucket/data/release_data.csv", header = True)

In [ ]:
df.show()

+----------+-------+------+----------+------+
|release_id|country|  year|     genre|format|
+----------+-------+------+----------+------+
|         1| Sweden|1999.0|Electronic| Vinyl|
|         2| Sweden|1998.0|Electronic| Vinyl|
|         3|     US|1999.0|Electronic|    CD|
|         4|     US|1999.0|Electronic|    CD|
|         5|Germany|1995.0|Electronic|    CD|
|         6| Sweden|1998.0|Electronic| Vinyl|
|         7|     US|2000.0|Electronic| Vinyl|
|         8|     US|2000.0|Electronic| Vinyl|
|         9|     US|2000.0|Electronic| Vinyl|
|        10|     US|1999.0|Electronic| Vinyl|
|        11|     US|1999.0|Electronic| Vinyl|
|        12|     US|1999.0|Electronic| Vinyl|
|        13|     US|2000.0|Electronic| Vinyl|
|        14|     US|1999.0|Electronic| Vinyl|
|        15|     US|2000.0|Electronic| Vinyl|
|        16|     US|2000.0|Electronic| Vinyl|
|        17|     US|1999.0|Electronic| Vinyl|
|        18|     US|2000.0|Electronic| Vinyl|
|        19| Canada|2000.0|Electro

In [ ]:
df.printSchema()

root
 |-- release_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- year: double (nullable = true)
 |-- genre: string (nullable = true)
 |-- format: string (nullable = true)



In [ ]:
columns = df.columns
print('columns in dataset: ', columns)

columns in dataset:  ['release_id', 'country', 'year', 'genre', 'format']


In [ ]:
total_rows = df.count()
print('total rows in dataset: ', total_rows)

total rows in dataset:  17372035


Null Values

In [ ]:
for column in df.columns:
  count = df.filter(df[column].isNull()).count()
  print(f'null values in {column}: {count} ({round(count/total_rows*100, 2)}%)')

null values in release_id: 0 (0.0%)
null values in country: 454549 (2.62%)
null values in year: 2164324 (12.46%)
null values in genre: 3 (0.0%)
null values in format: 0 (0.0%)


###Drop Null value

In [ ]:
df_clean = df.na.drop()

In [ ]:
for column in df_clean.columns:
  count = df_clean.filter(df[column].isNull()).count()
  print(f'null values in {column}: {count} ({round(count/total_rows*100, 2)}%)')

null values in release_id: 0 (0.0%)
null values in country: 0 (0.0%)
null values in year: 0 (0.0%)
null values in genre: 0 (0.0%)
null values in format: 0 (0.0%)


##unique values in dataset

How many Unique country in dataset

In [ ]:
total_country = df_clean.select('country').distinct().count()
print(f'total number of country is: {total_country}')

total number of country is: 278


Print Unique value in country

In [ ]:
distinct_values = df_clean.select('country').dropDuplicates().collect()
for value in distinct_values:
    print(value)

How many Unique Genre in dataset

In [ ]:
different_genre = df_clean.select('genre').distinct().count()
print(f'different genre is: {different_genre}')

different genre is: 15


different format in music

In [ ]:
different_format = df_clean.select('format').distinct().count()
print(f'different format in dataframe: {different_format}')

different format in dataframe: 55


##Number of release by country

In [ ]:
# Number of releases by country
release_by_country = (df.groupBy('country')
                        .count()
                        .orderBy(desc('count'))
                        .limit(20))

In [ ]:
# Collect the data to the driver node
data_country = release_by_country.collect()

count = [row['count'] for row in data_country]
country = [row['country'] for row in data_country]

d_country = dict()
d_country['country'] = country
d_country['count'] = count

In [ ]:
fig1 = px.bar(d_country, x='country', y='count', title='No. of releases by country')
fig1.show()

# Number of releases by year


In [ ]:
# Number of releases by country
release_by_year = (df_clean.groupBy('year')
                        .count()
                        .orderBy('year')
                      )

In [ ]:
# Collect the data to the driver node
data_year = release_by_year.collect()

year = [row['year'] for row in data_year]
count = [row['count'] for row in data_year]


d_year = dict()
d_year['year'] = year
d_year['count'] = count

In [ ]:
fig2 = px.line(d_year, x='year', y='count', title = "No. of releases by year")
fig2.show()

## Number of releases by genre

In [ ]:
# Number of releases by genre
release_by_genre = (df_clean.groupBy('genre')
                        .count()
                        .orderBy(desc('count'))
                        .limit(20))

In [ ]:
# Collect the data to the driver node
data_genre = release_by_genre.collect()

genre = [row['genre'] for row in data_genre]
count_genre = [row['count'] for row in data_genre]

d_genres = dict()
d_genres['genre'] = genre
d_genres['count_genre'] = count_genre

In [ ]:
fig3 = px.bar(d_genres, x='genre', y='count_genre', title = "No. of releases by genre")
fig3.show()


###Number of release by format

In [ ]:
#number of release by format
release_by_format = (df_clean.groupBy('format')
                        .count()
                        .orderBy(desc('count'))
                        .limit(20))

In [ ]:
# Collect the data to the driver node
data_format = release_by_format.collect()

format = [row['format'] for row in data_format]
count = [row['count'] for row in data_format]

d_format = dict()
d_format['format'] = format
d_format['count'] = count

In [ ]:
fig4 = px.bar(d_format, x='format', y='count', title = "No. of releases by format")
fig4.show()

###Genre Popularity throughout the countries

In [ ]:
release_by_genre_country = df_clean.groupBy('genre', 'country').agg(F.count('release_id').alias('count'))

# Add a new column with the rank of each group based on the count in descending order
release_by_genre_country = release_by_genre_country.withColumn(
    'rank', F.row_number().over(Window.partitionBy('country').orderBy(col('count').desc()))
)

# Select only the top-ranked genres for each country
genres = release_by_genre_country.filter(col('rank') <= 10)

In [ ]:
data_rbgc =  release_by_genre_country.collect()

genre = [row['genre'] for row in data_rbgc]
count = [row['count'] for row in data_rbgc]
country = [row['country'] for row in data_rbgc]

rbgc = dict()
rbgc['genre'] = genre
rbgc['count'] = count
rbgc['country'] = country

In [ ]:
fig_genre = px.bar(rbgc, x='genre', y='count', color='country',
              template='simple_white', title='Genre popularity throughout countries')

fig_genre.show()

###Genre popularity throughout the year




In [ ]:
# Group by 'genre' and 'year', and count the number of 'release_id' for each group
genre_year = df_clean.groupBy('genre', 'year').agg(F.count('release_id').alias('count'))

# Order the results by count in descending order
genre_year = genre_year.orderBy(F.desc('count'))



In [ ]:
data_tmp = genre_year.collect()

year = [row['year'] for row in data_tmp]
count = [row['count'] for row in data_tmp]
genre = [row['genre'] for row in data_tmp]

tmp = dict()
tmp['year'] = year
tmp['count'] = count
tmp['genre'] = genre

In [ ]:
fig_genre = px.bar(tmp, x='year', y='count', color='genre',
              template='simple_white', title='Genre popularity throughout year')

fig_genre.show()

### genre popularity within countries

In [ ]:
# Filter the PySpark DataFrame based on the country
genres = release_by_genre_country.filter(col('country').isin(countries))

In [ ]:
data_genres = genres.collect()

genre = [row['genre'] for row in data_genres]
count = [row['count'] for row in data_genres]
country = [row['country'] for row in data_genres]

genres_d = dict()
genres_d['genre'] = genre
genres_d['count'] = count
genres_d['country'] = country

In [ ]:
fig = px.bar(genres_d, x='country', y='count', color='genre',
             title='Genre popularity within countries', template='presentation')
fig.update_xaxes(tickangle=90)

fig.show()

###Formats throughout years

In [ ]:
# Group by 'format' and 'year', and count the number of 'release_id' for each group
format_year = df_clean.groupBy('format', 'year').agg(F.count('release_id').alias('count'))

# Order the results by count in descending order
format_year = format_year.orderBy(F.desc('count'))


In [ ]:
data_format_year = format_year.collect()

year = [row['year'] for row in data_format_year]
count = [row['count'] for row in data_format_year]
format = [row['format'] for row in data_format_year]

format_year_d = dict()
format_year_d['year'] = year
format_year_d['count'] = count
format_year_d['format'] = format

In [ ]:
figf = px.bar(format_year_d, x='year', y='count', color='format', template = 'plotly_white', title = 'Formats throughout years')
figf.show()